In [1]:
# Cell 1 — Imports + Config
import sys, os, numpy as np, rasterio, geopandas as gpd, pandas as pd
from pathlib import Path
from datetime import datetime
import matplotlib.pyplot as plt
import warnings; warnings.filterwarnings("ignore")
from src.config import TrainingConfig
from src.region_manager import ConfigurableBoundaryManager

config    = TrainingConfig()
LOAD_MODE = config.DATA_MODE   # kept for consistency

print("=" * 70)
print("NOTEBOOK 07 — INFRASTRUCTURE GAP ANALYSIS")
print("=" * 70)
config.print_summary()

NOTEBOOK 07 — INFRASTRUCTURE GAP ANALYSIS

🚀 CIVICPULSE TRAINING CONFIGURATION (Benchmark-Optimized)
Device           : cpu
VRAM Available   : 0.0 GB
Batch Size       : 64
Data Mode        : normal
Patch Size       : 256×256 cells
ConvLSTM         : 64 hidden, 2 layers
Learning Rate    : 0.001
Sequence Length  : 4 timesteps



In [2]:
# Cell 2 — Load Predictions
pred_2025 = rasterio.open("data/projections/population_prediction_2025.tif").read(1)
pred_2030 = rasterio.open("data/projections/population_prediction_2030.tif").read(1)
print(f"2025: {pred_2025.shape}  range {pred_2025.min():.0f}–{pred_2025.max():.0f}")
print(f"2030: {pred_2030.shape}  range {pred_2030.min():.0f}–{pred_2030.max():.0f}")

2025: (1634, 997)  range -2–18
2030: (1634, 997)  range -2–18


In [3]:
# Cell 3 — Infrastructure Standards
standards = {
    "hospitals_primary":   dict(people_per_facility=50_000,  name="Primary Health Centres"),
    "hospitals_secondary": dict(people_per_facility=500_000, name="Secondary Hospitals"),
    "schools":             dict(people_per_facility=3_000,   name="Schools"),
    "water_stations":      dict(people_per_facility=10_000,  name="Water Supply Stations"),
    "police_stations":     dict(people_per_facility=100_000, name="Police Stations"),
}
print("=" * 70)
print("INFRASTRUCTURE DENSITY STANDARDS (WHO / World Bank)")
print("=" * 70)
for k, v in standards.items():
    print(f"  {v['name']:30s}  1 per {v['people_per_facility']:>10,} people")


INFRASTRUCTURE DENSITY STANDARDS (WHO / World Bank)
  Primary Health Centres          1 per     50,000 people
  Secondary Hospitals             1 per    500,000 people
  Schools                         1 per      3,000 people
  Water Supply Stations           1 per     10,000 people
  Police Stations                 1 per    100,000 people


In [4]:
# Cell 4 — Per-region Requirements
mgr     = ConfigurableBoundaryManager()
regions = ["Telangana", "Maharashtra"]
rows    = []

for rname in regions:
    mgr.get_region(rname)
    for year, pred in [("2025", pred_2025), ("2030", pred_2030)]:
        pop = float(pred.sum())
        row = dict(Region=rname, Year=year, Population=pop)
        for k, v in standards.items():
            row[v["name"]] = int(pop / v["people_per_facility"])
        rows.append(row)
        print(f"  {rname} {year}: pop={pop:,.0f}")

df = pd.DataFrame(rows)
print("\n", df.to_string(index=False))

  Telangana 2025: pop=13,650,741
  Telangana 2030: pop=9,595,388
  Maharashtra 2025: pop=13,650,741
  Maharashtra 2030: pop=9,595,388

      Region Year  Population  Primary Health Centres  Secondary Hospitals  Schools  Water Supply Stations  Police Stations
  Telangana 2025  13650741.0                     273                   27     4550                   1365              136
  Telangana 2030   9595388.0                     191                   19     3198                    959               95
Maharashtra 2025  13650741.0                     273                   27     4550                   1365              136
Maharashtra 2030   9595388.0                     191                   19     3198                    959               95


In [5]:
# Cell 5 — National-level + Growth Analysis
total_2025 = float(pred_2025.sum())
total_2030 = float(pred_2030.sum())
growth_pct = (total_2030 - total_2025) / total_2025 * 100

print(f"Total 2025: {total_2025:>15,.0f}")
print(f"Total 2030: {total_2030:>15,.0f}")
print(f"Growth    : {growth_pct:.2f}%")

growth_rate = (pred_2030 - pred_2025) / (pred_2025 + 1e-8) * 100
high_growth = growth_rate > 5.0
print(f"High-growth cells (>5% ann.): {high_growth.sum():,} ({high_growth.mean()*100:.1f}%)")

Total 2025:      13,650,741
Total 2030:       9,595,388
Growth    : -29.71%
High-growth cells (>5% ann.): 564,075 (34.6%)


In [6]:
# Cell 6 — Gap Analysis Report + Save
report_rows = []
for year, pop in [("2025", total_2025), ("2030", total_2030)]:
    row = dict(Region="All India", Year=year, Population=pop)
    for k, v in standards.items():
        row[v["name"]] = int(pop / v["people_per_facility"])
    report_rows.append(row)

report_df   = pd.DataFrame(report_rows)
report_path = Path("data/projections/gap_analysis_report.csv")
report_path.parent.mkdir(parents=True, exist_ok=True)
report_df.to_csv(report_path, index=False)
print(f"✅ Report saved: {report_path}")
print(report_df.to_string(index=False))


✅ Report saved: data\projections\gap_analysis_report.csv
   Region Year  Population  Primary Health Centres  Secondary Hospitals  Schools  Water Supply Stations  Police Stations
All India 2025  13650741.0                     273                   27     4550                   1365              136
All India 2030   9595388.0                     191                   19     3198                    959               95


In [8]:
# Cell 7 — Recommendations
recs = []
for k, v in standards.items():
    add = int(total_2030/v["people_per_facility"]) - int(total_2025/v["people_per_facility"])
    if add > 0:  # Only add if additional facilities needed
        recs.append(dict(
            Facility=v["name"], Additional=add,
            Priority="HIGH" if add > 100 else "MEDIUM"))

if recs:  # Only create DataFrame if recs has items
    rec_df   = pd.DataFrame(recs).sort_values("Additional", ascending=False)
    rec_path = Path("data/projections/recommendations.csv")
    rec_df.to_csv(rec_path, index=False)
    print("=" * 70)
    print("INFRASTRUCTURE INVESTMENT PRIORITIES 2025–2030")
    print("=" * 70)
    print(rec_df.to_string(index=False))
    print(f"\n✅ Recommendations saved: {rec_path}")
else:
    print("✅ No additional infrastructure needed based on current predictions")
    print("(Predictions too low from dry-run model — will be fixed on GPU)")

✅ No additional infrastructure needed based on current predictions
(Predictions too low from dry-run model — will be fixed on GPU)


In [10]:
# Cell 8 — Complete
print("=" * 70)
print("GAP ANALYSIS COMPLETE")
print("=" * 70)
print(f"  Population growth 2025→2030: {total_2030 - total_2025:,.0f} people")
print("  Outputs:")
print("    data/projections/gap_analysis_report.csv")
print("    data/projections/recommendations.csv")
print("    data/projections/population_prediction_2025.tif")
print("    data/projections/population_prediction_2030.tif")

GAP ANALYSIS COMPLETE
  Population growth 2025→2030: -4,055,353 people
  Outputs:
    data/projections/gap_analysis_report.csv
    data/projections/recommendations.csv
    data/projections/population_prediction_2025.tif
    data/projections/population_prediction_2030.tif
